# Create Dataset for employees

In [20]:
!curl "https://api.mockaroo.com/api/954b36e0?count=100&key=7a2967f0" > "employee.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5746    0  5746    0     0   5763      0 --:--:-- --:--:-- --:--:--  5763


## Setup database

In [21]:
import sqlite3
import pandas as pd
import os

In [34]:
# Define SQL schemas for creating tables

employee_schema = """
CREATE TABLE IF NOT EXISTS employee (
    e_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    Salary INT,
    department VARCHAR(50)
);
"""

In [49]:
db_name = 'emp.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

Removed existing database 'emp.db'.


In [50]:
import sqlite3
import pandas as pd
import os



COLUMN_DATA_TYPES = {
    'employee': {
        'e_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'Salary': 'int64',
        'department': 'object'
    }
}


# --- Database setup ---
db_name = 'emp.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(employee_schema)

    print("Tables 'employee' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/employee.csv': 'employee'

    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'emp.db' created and connected successfully. ✅
Tables 'employee' created successfully.

Processing '/content/employee.csv' for table 'employee'...
  -> Data from '/content/employee.csv' loaded into 'employee' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


# Tangent 2: You should setup your free API Key using Google's AI Studio

https://aistudio.google.com/


In [14]:
!pip install google-genai

In [15]:
from google import genai
from google.colab import userdata

genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

# Tangent 3: You should learn about Prompt Engineering

In [42]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`employee` table:**

```sql
CREATE TABLE employee (
    e_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    Salary INT,
    department VARCHAR(50)
);
```


-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all employee from Legal department"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM employee WHERE department = 'Legal';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 employee with the highest Salary?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT first_name, last_name, Salary FROM employee ORDER BY Salary DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me some departments"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'some' means? For example, '2 departments', '3 departments', or '5 departments'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [43]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


In [52]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='emp.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [53]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [59]:
text2sql(genai_client, prompt, "Show me all the department with count")

Input Token Count: 1128
Thoughts Token Count: 41
Output Token Count: 37
Total Token Count: 1206

Executing query on 'emp.db':
SELECT department, COUNT(*) AS employee_count FROM employee GROUP BY department;
Query executed successfully.


,department,employee_count
0,Accounting,10
1,Business Development,8
2,Engineering,8
3,Human Resources,9
4,Legal,15
5,Marketing,2
6,Product Management,9
7,Research and Development,9
8,Sales,7
9,Services,9


In [65]:
text2sql(genai_client, prompt, "Show me 3 emplotees from training department with highest Salay")

Input Token Count: 1135
Thoughts Token Count: 119
Output Token Count: 48
Total Token Count: 1302

Executing query on 'emp.db':
SELECT first_name, last_name, Salary FROM employee WHERE department = 'Training' ORDER BY Salary DESC LIMIT 3;
Query executed successfully.


,first_name,last_name,Salary
0,Aurilia,Macrow,89769
1,Taylor,Dilloway,53673
2,Sue,Beaudry,962


In [64]:
text2sql(genai_client, prompt, "Show me 10 emplotees with Highest Salay")

Input Token Count: 1133
Thoughts Token Count: 251
Output Token Count: 43
Total Token Count: 1427

Executing query on 'emp.db':
SELECT first_name, last_name, Salary FROM employee ORDER BY Salary DESC LIMIT 10;
Query executed successfully.


,first_name,last_name,Salary
0,Emmery,Ikin,92592
1,Aurilia,Macrow,89769
2,Gilligan,Jurgensen,85852
3,Morrie,Tams,73665
4,Elaina,Lemon,72140
5,Truda,Merrywether,71976
6,Selene,Eveleigh,67452
7,Egon,Colbourn,65408
8,Concordia,Guisot,64223
9,Estele,Drees,59330


## Conclusion
Based on the steps we've taken, we have successfully:

- Created a SQLite database and loaded employee data from a CSV file.
- Defined a detailed prompt for a natural language to SQL model, specializing in SQLite and the provided employee schema.
- Demonstrated the use of a text-to-SQL function that takes a user query, uses the prompt and a generative model to generate an SQL query, and then executes that query against the database.
- Successfully executed several example queries to retrieve information about employee departments and salaries.

This notebook provides a working example of how to build a text-to-SQL interface for a simple database using generative AI.